In [85]:
#!/usr/bin/env python
# coding: utf-8

# In[13]:


import pandas as pd
import numpy as np
import os
from ast import literal_eval

pd.options.mode.chained_assignment = None


# In[14]:


data_path = "./data"
if os.listdir(data_path) != ['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']:
    print("[ERROR] Please download and unzip the dataset in a subdirectory './data'.")
else:
    print("[INFO] The dataset is correctly placed.")


# ### Loading the data

# In[15]:


tmdb_credits = pd.read_csv(os.path.join(data_path, "tmdb_5000_credits.csv"))
tmdb_credits.head(2)


# In[16]:


tmdb_movies = pd.read_csv(os.path.join(data_path, "tmdb_5000_movies.csv"))
tmdb_movies.head(2)


# ### Cleaning the datasets
# #### Cleaning tmdb credits

# In[17]:


literal_eval(tmdb_credits.cast[0])

def mapping_casts(list_casts, useful_keys = {"id", "gender", "name"}):
    """Mapping casts to keep useful information."""
    result = []
    for cast in list_casts:
        filtering = {k:v for k,v in cast.items() if k in useful_keys}
        if len(filtering) == len(useful_keys):
            result.append(filtering)
    return result
mapping_casts(literal_eval(tmdb_credits.cast[0]))[:5]


# In[18]:


L = literal_eval(tmdb_credits.crew[0])

def mapping_crew(list_crew, useful_keys = {"id", "job", "name", "gender"}, keeping_jobs = {"Original Music Composer", "Director"}):
    """Mapping crew to keep useful information."""
    result = []
    for person in list_crew:
        filtering = {k:v for k,v in person.items() if (k in useful_keys and person["job"] in keeping_jobs)}
        if filtering != {} and len(filtering) == len(useful_keys):            
            result.append(filtering)
    return result

mapping_crew(L)


# In[19]:


tmdb_credits_clean = tmdb_credits.copy()
tmdb_credits_clean['cast'] = tmdb_credits_clean['cast'].apply(lambda x: mapping_casts(literal_eval(x)))
tmdb_credits_clean['crew'] = tmdb_credits_clean['crew'].apply(lambda x: mapping_crew(literal_eval(x)))
tmdb_credits_clean.head()


# In[20]:


tmdb_credits_clean_cast = tmdb_credits_clean[["movie_id", "title", "cast"]].explode("cast")

def get_gender(cast):
    splitted = str(cast).split()
    if len(splitted)>1:
        return splitted[1][0]
    else:
        -1
        
def get_id(cast):
    splitted = str(cast).split()
    if len(splitted)>1:
        return splitted[3].replace(",", "")
    else:
        -1
        
def get_name(cast):
    splitted = str(cast).split(", ")
    if len(splitted)>1:
        return splitted[2].replace("'", "").replace(",", "").replace("name: ", "").replace("}", "")
    else:
        -1

tmdb_credits_clean_cast['gender'] = tmdb_credits_clean_cast.cast.apply(get_gender)
tmdb_credits_clean_cast = tmdb_credits_clean_cast[tmdb_credits_clean_cast.gender != -1]

tmdb_credits_clean_cast['person_id'] = tmdb_credits_clean_cast.cast.apply(get_id)
tmdb_credits_clean_cast = tmdb_credits_clean_cast[tmdb_credits_clean_cast.person_id != -1]

tmdb_credits_clean_cast['name'] = tmdb_credits_clean_cast.cast.apply(get_name)
tmdb_credits_clean_cast = tmdb_credits_clean_cast[tmdb_credits_clean_cast.name != -1]

tmdb_credits_clean_cast['job'] = "actor"
tmdb_credits_clean_cast.drop(columns=['cast'], inplace=True)
tmdb_credits_clean_cast.head()


# In[21]:


tmdb_credits_clean_crew = tmdb_credits_clean[["movie_id", "title", "crew"]].explode("crew")

        
def get_name_crew(crew):
    splitted = str(crew).split(': ')
    if len(splitted)>1:
        return splitted[4].replace(",", "").replace("'", "").replace("}", "")
    else:
        -1
        
def get_job(crew):
    splitted = str(crew).split(" '")
    if len(splitted)>1:
        return splitted[3].replace("'", "").replace(",", "")
    else:
        -1
    



tmdb_credits_clean_crew['gender'] = tmdb_credits_clean_crew.crew.apply(get_gender)
tmdb_credits_clean_crew = tmdb_credits_clean_crew[tmdb_credits_clean_crew.gender != -1]

tmdb_credits_clean_crew['person_id'] = tmdb_credits_clean_crew.crew.apply(get_id)
tmdb_credits_clean_crew = tmdb_credits_clean_crew[tmdb_credits_clean_crew.person_id != -1]

tmdb_credits_clean_crew['name'] = tmdb_credits_clean_crew.crew.apply(get_name_crew)
tmdb_credits_clean_crew = tmdb_credits_clean_crew[tmdb_credits_clean_crew.name != -1]

tmdb_credits_clean_crew['job'] = tmdb_credits_clean_crew.crew.apply(get_job)
tmdb_credits_clean_crew = tmdb_credits_clean_crew[tmdb_credits_clean_crew.job != -1]
tmdb_credits_clean_crew.drop(columns=['crew'], inplace=True)
tmdb_credits_clean_crew.head()


[INFO] The dataset is correctly placed.


,movie_id,title,gender,person_id,name,job
0,19995,Avatar,2,1729,James Horner,Original Music Composer
0,19995,Avatar,2,2710,James Cameron,Director
1,285,Pirates of the Caribbean: At World's End,2,1704,Gore Verbinski,Director
1,285,Pirates of the Caribbean: At World's End,2,947,Hans Zimmer,Original Music Composer
2,206647,Spectre,2,153,Thomas Newman,Original Music Composer


In [86]:
tmdb_credits_clean_cast

,movie_id,title,gender,person_id,name,job
0,19995,Avatar,2,65731,Sam Worthington,actor
0,19995,Avatar,1,8691,Zoe Saldana,actor
0,19995,Avatar,1,10205,Sigourney Weaver,actor
0,19995,Avatar,2,32747,Stephen Lang,actor
0,19995,Avatar,1,17647,Michelle Rodriguez,actor
...,...,...,...,...,...,...
4802,25975,My Date with Drew,2,21315,Eric Roberts,actor
4802,25975,My Date with Drew,0,2171,Griffin Dunne,actor
4802,25975,My Date with Drew,2,2231,Samuel L. Jackson,actor
4802,25975,My Date with Drew,2,14407,Matt LeBlanc,actor


In [87]:
tmdb_credits_clean_crew

,movie_id,title,gender,person_id,name,job
0,19995,Avatar,2,1729,James Horner,Original Music Composer
0,19995,Avatar,2,2710,James Cameron,Director
1,285,Pirates of the Caribbean: At World's End,2,1704,Gore Verbinski,Director
1,285,Pirates of the Caribbean: At World's End,2,947,Hans Zimmer,Original Music Composer
2,206647,Spectre,2,153,Thomas Newman,Original Music Composer
...,...,...,...,...,...,...
4800,231617,"Signed, Sealed, Delivered",0,1219158,Scott Smith,Director
4801,126186,Shanghai Calling,2,208138,Daniel Hsia,Director
4802,25975,My Date with Drew,2,85563,Brian Herzlinger,Director
4802,25975,My Date with Drew,2,94471,Jon Gunn,Director


In [88]:
tmdb_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [135]:
predicates_to_check = [#lambda data : (data.vote_count > data.vote_count.describe()['25%']), 
                       lambda data : (data.budget > 1),
                       lambda data : (data.revenue > 1000),
                        lambda data : (data.popularity > 1)]

In [136]:
tmdb_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [137]:
tmdb_mov = tmdb_movies.copy()
for p in predicates_to_check:
    tmdb_mov = tmdb_mov[p(tmdb_mov)]
    
tmdb_mov.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [138]:
tm = tmdb_mov

In [139]:
from datetime import datetime

In [140]:
tm.release_date = tm.release_date.apply(lambda d : datetime(*tuple([int(x) for x in d.split('-')])))

In [141]:
tm['realease_year'] = tm.release_date.apply(lambda d : d.year)

In [142]:
import ast

tm.genres = tm.genres.apply(lambda d : [x['name'] for x in ast.literal_eval(d)])

In [143]:
import plotly.express as px
df = px.data.gapminder()

fig = px.scatter(tm.explode('genres').groupby(['realease_year', 'genres']).mean().reset_index().query("realease_year==2016"), x="genres", y="revenue",
	         size="popularity", color="popularity",
                log_y=True, size_max=60, title="2016")
fig.show()

In [144]:
tm.revenue.describe()

count    3.159000e+03
mean     1.238399e+08
std      1.875258e+08
min      6.399000e+03
25%      1.837731e+07
50%      5.777711e+07
75%      1.494847e+08
max      2.787965e+09
Name: revenue, dtype: float64

In [145]:
#https://plotly.com/python/animations/
df = tm.copy()
df = df[df.genres.apply(lambda x : len(x)>0)]
df.genres = df.genres.apply(lambda x : x[0])

In [146]:
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,realease_year
0,237000000,Action,http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,2009
1,300000000,Adventure,http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007
2,245000000,Action,http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,2015
3,250000000,Action,http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,2012
4,260000000,Action,http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,31192,Drama,http://downterrace.blogspot.com/,42151,"[{""id"": 9826, ""name"": ""murder""}, {""id"": 10123,...",en,Down Terrace,After serving jail time for a mysterious crime...,1.330379,[],...,2009-09-01,10000,89.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You're only as good as the people you know.,Down Terrace,6.3,26,2009
4773,27000,Comedy,http://www.miramax.com/movie/clerks/,2292,"[{""id"": 1361, ""name"": ""salesclerk""}, {""id"": 30...",en,Clerks,Convenience and video store clerks Dante and R...,19.748658,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...",...,1994-09-13,3151130,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Just because they serve you doesn't mean they ...,Clerks,7.4,755,1994
4788,12000,Horror,NaN,692,"[{""id"": 237, ""name"": ""gay""}, {""id"": 900, ""name...",en,Pink Flamingos,Notorious Baltimore criminal and underground f...,4.553644,"[{""name"": ""Dreamland Productions"", ""id"": 407}]",...,1972-03-12,6000000,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,An exercise in poor taste.,Pink Flamingos,6.2,110,1972
4796,7000,Science Fiction,http://www.primermovie.com,14337,"[{""id"": 1448, ""name"": ""distrust""}, {""id"": 2101...",en,Primer,Friends/fledgling entrepreneurs invent a devic...,23.307949,"[{""name"": ""Thinkfilm"", ""id"": 446}]",...,2004-10-08,424760,77.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,What happens if it actually works?,Primer,6.9,658,2004


In [147]:
df.budget.describe()

count    3.159000e+03
mean     4.142912e+07
std      4.456172e+07
min      1.000000e+01
25%      1.100000e+07
50%      2.600000e+07
75%      5.500000e+07
max      3.800000e+08
Name: budget, dtype: float64

In [148]:
import plotly.express as px

px.scatter(df.query('genres=="Action"'), x="revenue", y="popularity", animation_frame="realease_year",
           size="budget", color='genres',
                log_x=True, log_y=True, range_x=[100,10000000000], range_y=[1,1000], size_max=60, title="2016",
            )

In [149]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,realease_year
0,237000000,Action,http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,2009
1,300000000,Adventure,http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007


In [150]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'realease_year'],
      dtype='object')

In [151]:
df[['revenue','popularity', 'budget', 'original_title', 'genres', 'realease_year']].to_csv("./jeremy/bubbleData.csv")

In [64]:
for v in df.genres.unique():
    print( "<option value=\""+v+"\">"+v+"</option>")

<option value="Action">Action</option>
<option value="Adventure">Adventure</option>
<option value="Fantasy">Fantasy</option>
<option value="Animation">Animation</option>
<option value="Science Fiction">Science Fiction</option>
<option value="Drama">Drama</option>
<option value="Thriller">Thriller</option>
<option value="Family">Family</option>
<option value="Comedy">Comedy</option>
<option value="History">History</option>
<option value="War">War</option>
<option value="Western">Western</option>
<option value="Romance">Romance</option>
<option value="Crime">Crime</option>
<option value="Mystery">Mystery</option>
<option value="Horror">Horror</option>
<option value="Music">Music</option>
<option value="Documentary">Documentary</option>


In [65]:
len(df.genres.unique())

18

In [117]:
sorted(df.realease_year.unique())

[1916,
 1925,
 1927,
 1929,
 1930,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1939,
 1940,
 1941,
 1942,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016]